In [1]:
import tempfile
import pyabc
import pandas as pd

import os
import sys
sys.path.insert(0, os.path.abspath("../../../"))
import asteca

# Load the observed cluster with pandas
cluster_df = pd.read_csv("../_static/cluster.csv")

my_cluster = asteca.cluster(
    cluster_df=cluster_df,
    magnitude="Gmag",
    e_mag="e_Gmag",
    color="BP-RP",
    e_color='e_BP-RP',
)

# Isochrones parameters
isochs = asteca.isochrones(
    isochs_path="../_static/parsec/",
    magnitude={"Gmag": 6390.21},
    color={"G_BPmag": 5182.58, "G_RPmag": 7825.08},
)

# Synthetic clusters parameters
synthcl = asteca.synthetic(isochs)

fix_params = {"alpha": 0.09, "beta": 0.94, "Rv": 3.1, "DR": 0.}
synthcl.calibrate(my_cluster, fix_params)

# Instantiate the likelihood
likelihood = asteca.likelihood(my_cluster)

def model(fit_params):
    """Generate synthetic cluster"""
    synth_clust = synthcl.generate(fit_params)
    # pyABC expects a dictionary from this function
    synth_dict = {"data": synth_clust}
    return synth_dict

max_lkl = likelihood.max_lkl
def distance(synth_dict, _):
    """
    The likelihood is maximized for better fitted models but this distance requires
    minimization. Hence we normalize and invert.
    """
    lkl = likelihood.get(synth_dict["data"])
    return 1 - lkl / max_lkl

met_min, met_max, loga_min, loga_max = synthcl.min_max()
# Define a pyABC Distribution(). Uniform distributions are employed for all the parameters
# here but the user can of course change this as desired. See the pyABC docs for more
# information.
priors = pyabc.Distribution(
    {
        "met": pyabc.RV("uniform", met_min, met_max - met_min),
        "loga": pyabc.RV("uniform", loga_min, loga_max - loga_min),
        "dm": pyabc.RV("uniform", 7, 10 - 7),
        "Av": pyabc.RV("uniform", 0, 2 - 0)
    }
)

Reading and processing cluster data
Cluster object generated

Processing PARSEC isochrones for 1 photometric systems, N_met=1, N_age=10...
Isochrone object generated

Setting random seed to 17658
Sampling selected IMF (chabrier_2014)
Generating binary data
Obtaining extinction coefficients
Synthetic clusters object generated

Likelihood object generated



In [2]:
abc = pyabc.ABCSMC(model, priors, distance, population_size=100)

db_path = os.path.join(tempfile.gettempdir(), "test.db")
abc.new("sqlite:///" + db_path)

ABC.Sampler INFO: Parallelize sampling on 8 processes.
ABC.History INFO: Start <ABCSMC id=17, start_time=2024-04-16 17:17:39>


In [3]:
history = abc.run(minimum_epsilon=0.01, max_nr_populations=5)

ABC INFO: Calibration sample t = -1.
ABC INFO: t: 0, eps: 1.78425698e-01.
/home/gabriel/miniconda3/envs/asteca/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:1988: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/home/gabriel/miniconda3/envs/asteca/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:1988: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/home/gabriel/miniconda3/envs/asteca/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:1988: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/home/gabriel/miniconda3/envs/asteca/lib/python3.11/site-packages/scipy/stats/_distn_infrastructure.py:1988: RuntimeWarning: invalid value encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/home/gabriel/miniconda3/envs/asteca/lib/python3.11/site-packages/scipy/stats/_distn_infrastructur

AssertionError: The population total weight nan is not normalized.